In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import os
import mlflow
import mlflow.sklearn
import boto3




# Set AWS credentials and region directly
aws_access_key_id = ''
aws_secret_access_key = ''
aws_region = ''  # replace with your region


session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)

# Set environment variables for AWS credentials and region
os.environ["AWS_ACCESS_KEY_ID"] = aws_access_key_id
os.environ["AWS_SECRET_ACCESS_KEY"] = aws_secret_access_key
os.environ["AWS_DEFAULT_REGION"] = aws_region


# Set the S3 bucket as the default artifact root
s3_bucket = 'mlflow-scenario-exp-tracking'
mlflow.set_tracking_uri("url")  # replace with your MLflow tracking server URI if different
mlflow.set_experiment("my-new-s3-bucket")  # set your experiment name


with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")
    mlflow.sklearn.log_model(lr, artifact_path="models")


2024/06/06 14:51:34 INFO mlflow.tracking.fluent: Experiment with name 'my-new-s3-bucket' does not exist. Creating a new experiment.


default artifacts URI: 's3://mlflow-scenario-exp-tracking/5/da4ee26b394f49ff8a93f46309087536/artifacts'


In [3]:
mlflow.search_experiments()


[<Experiment: artifact_location='s3://mlflow-scenario-exp-tracking/5', creation_time=1717671094460, experiment_id='5', last_update_time=1717671094460, lifecycle_stage='active', name='my-new-s3-bucket', tags={}>,
 <Experiment: artifact_location='s3://mlflow-scenario-exp-tracking/4', creation_time=1717669874891, experiment_id='4', last_update_time=1717669874891, lifecycle_stage='active', name='my-scenario-news3bucket', tags={}>,
 <Experiment: artifact_location='s3://exp-mlflow-artifacts-bucket/3', creation_time=1717668732842, experiment_id='3', last_update_time=1717668732842, lifecycle_stage='active', name='my-experiment-scenario3', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-bucket/2', creation_time=1717663857220, experiment_id='2', last_update_time=1717663857220, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-bucket/1', creation_time=1717663577118, experiment_id='1', last_update_time=1717663577118,

Interacting with the model registry

In [4]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{"url"}:5000")

In [6]:
client.search_registered_models()

[]

In [7]:
run_id = client.search_runs(experiment_ids='5')[0].info.run_id

mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/06/06 15:04:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1717671839484, current_stage='None', description='', last_updated_timestamp=1717671839484, name='iris-classifier', run_id='da4ee26b394f49ff8a93f46309087536', run_link='', source='s3://mlflow-scenario-exp-tracking/5/da4ee26b394f49ff8a93f46309087536/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>